In [1]:
import pandas as pd
import csv
import os

In [12]:
data_path = f"{os.path.dirname(os.getcwd())}/data/project_health/monthly_closed_issues_2mo"
datafiles = [f for f in os.listdir(data_path) if f.endswith("csv")]
datafiles

['health_project0000.csv',
 'health_project0001.csv',
 'health_project0002.csv',
 'health_project0003.csv',
 'health_project0004.csv',
 'health_project0005.csv',
 'health_project0006.csv',
 'health_project0007.csv',
 'health_project0008.csv',
 'health_project0009.csv',
 'health_project0010.csv',
 'health_project0011.csv']

In [13]:
for d in datafiles:
    df = pd.read_csv(f"{data_path}/{d}")
    
    break

In [14]:
df

,monthly_commits,monthly_contributors,monthly_open_PRs,monthly_closed_PRs,monthly_merged_PRs,monthly_PR_mergers,monthly_PR_comments,monthly_open_issues,monthly_closed_issues,monthly_issue_comments,monthly_stargazer,monthly_forks,monthly_watchers,monthly_closed_issues_2mo
0,123,1,0,0,0,0,0,10,23,23,0,0,0,5
1,52,1,0,0,0,0,0,2,3,0,0,0,0,0
2,40,1,0,0,0,0,0,1,5,0,0,0,0,0
3,7,1,0,0,0,0,0,0,0,0,0,0,2,5
4,1,1,0,0,0,0,0,0,0,0,0,0,3,0
5,21,1,1,0,1,1,0,0,5,1,0,0,2,4
6,0,0,0,0,0,0,0,0,0,0,0,0,4,3
7,23,1,0,0,0,0,0,1,4,6,0,0,1,24
8,34,1,0,0,0,0,0,0,3,0,0,0,1,21
9,128,2,6,0,6,1,0,6,24,1,0,0,2,3
